<a href="https://colab.research.google.com/github/oddrationale/AdventOfCode2023Python/blob/main/Day10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<article class="day-desc"><h2>--- Day 10: Pipe Maze ---</h2><p>You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.</p>
<p>You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "<a href="https://en.wikipedia.org/wiki/Hot_spring" target="_blank">Hot Springs</a>" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.</p>
<p>The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.</p>
<p>Scanning the area, you discover that the entire field you're standing on is <span title="Manufactured by Hamilton and Hilbert Pipe Company">densely packed with pipes</span>; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).</p>
<p>The pipes are arranged in a two-dimensional grid of <em>tiles</em>:</p>
<ul>
<li><code>|</code> is a <em>vertical pipe</em> connecting north and south.</li>
<li><code>-</code> is a <em>horizontal pipe</em> connecting east and west.</li>
<li><code>L</code> is a <em>90-degree bend</em> connecting north and east.</li>
<li><code>J</code> is a <em>90-degree bend</em> connecting north and west.</li>
<li><code>7</code> is a <em>90-degree bend</em> connecting south and west.</li>
<li><code>F</code> is a <em>90-degree bend</em> connecting south and east.</li>
<li><code>.</code> is <em>ground</em>; there is no pipe in this tile.</li>
<li><code>S</code> is the <em>starting position</em> of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.</li>
</ul>
<p>Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is <em>one large, continuous loop</em>.</p>
<p>For example, here is a square loop of pipe:</p>
<pre><code>.....
.F-7.
.|.|.
.L-J.
.....
</code></pre>
<p>If the animal had entered this loop in the northwest corner, the sketch would instead look like this:</p>
<pre><code>.....
.<em>S</em>-7.
.|.|.
.L-J.
.....
</code></pre>
<p>In the above diagram, the <code>S</code> tile is still a 90-degree <code>F</code> bend: you can tell because of how the adjacent pipes connect to it.</p>
<p>Unfortunately, there are also many pipes that <em>aren't connected to the loop</em>! This sketch shows the same loop as above:</p>
<pre><code>-L|F7
7S-7|
L|7||
-L-J|
L|-JF
</code></pre>
<p>In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to <code>S</code>, pipes those pipes connect to, pipes <em>those</em> pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including <code>S</code>, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).</p>
<p>Here is a sketch that contains a slightly more complex main loop:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:</p>
<pre><code>7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
</code></pre>
<p>If you want to <em>get out ahead of the animal</em>, you should find the tile in the loop that is <em>farthest</em> from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps <em>along the loop</em> to reach from the starting point - regardless of which way around the loop the animal went.</p>
<p>In the first example with the square loop:</p>
<pre><code>.....
.S-7.
.|.|.
.L-J.
.....
</code></pre>
<p>You can count the distance each tile in the loop is from the starting point like this:</p>
<pre><code>.....
.012.
.1.3.
.23<em>4</em>.
.....
</code></pre>
<p>In this example, the farthest point from the start is <code><em>4</em></code> steps away.</p>
<p>Here's the more complex loop again:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here are the distances for each tile on that loop:</p>
<pre><code>..45.
.236.
01.7<em>8</em>
14567
23...
</code></pre>
<p>Find the single giant loop starting at <code>S</code>. <em>How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?</em></p>
</article>

In [1]:
from google.colab import drive
drive.mount(r"/content/drive")

input_file = r"/content/drive/MyDrive/Colab Notebooks/AoC2023/input/10.txt"
with open(input_file, "r") as file:
    input = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from collections import deque

In [3]:
def parse_input(input: str) -> list[str]:
    """
    Parse the multiline puzzle input into a list of strings.

    Each line of the input is converted into a string in the list.

    Args:
    input (str): A multiline string representing the puzzle input.

    Returns:
    list[str]: A list of strings, each representing a line from the input.
    """
    # Split the input string into lines
    return input.splitlines()

In [4]:
def find_start_position(grid: list[str]) -> tuple[int, int]:
    """
    Find the coordinates of the starting position 'S' in the grid.

    Args:
    grid (list[str]): A list of strings representing the pipe grid.

    Returns:
    tuple[int, int]: The coordinates (row, column) of the starting position 'S'.
                      Returns (-1, -1) if 'S' is not found.
    """
    for row_index, row in enumerate(grid):
        if 'S' in row:
            return (row_index, row.index('S'))
    return (-1, -1)  # Return an invalid position if 'S' is not found

In [5]:
def bfs_furthest_distance(grid: list[str], start: tuple[int, int]) -> tuple[int, set[tuple[int, int]]]:
    """
    Find the furthest distance in a pipe loop using BFS.

    Args:
    grid (list[str]): The grid representing the pipe layout.
    start (tuple[int, int]): The starting position (row, column).

    Returns:
    tuple[int, set[tuple[int, int]]]:
        The number of steps to the furthest point in the loop from the start.
        The set of tuples of the coordinates of the pipes in the path.
    """

    # Define the directions each pipe type can connect to
    connections = {
        '|': {'U', 'D'},
        '-': {'L', 'R'},
        'L': {'U', 'R'},
        'J': {'L', 'U'},
        '7': {'L', 'D'},
        'F': {'R', 'D'},
        'S': {'U', 'D', 'L', 'R'},  # 'S' can connect in any direction
        '.': set(),
    }

    # Define the offsets for moving in each direction
    offsets = {
        'U': (-1, 0),
        'D': (1, 0),
        'L': (0, -1),
        'R': (0, 1)
    }

    def is_valid_move(from_pipe: str, to_pipe: str, direction: str) -> bool:
        """Check if the move is valid based on the pipe connections."""
        opposite = {'U': 'D', 'D': 'U', 'L': 'R', 'R': 'L'}
        return opposite[direction] in connections[to_pipe]

    # Initialize the BFS queue and visited set
    # Each element in the queue is a tuple (position, distance)
    queue: deque[tuple[tuple[int, int], int]] = deque([(start, 0)])
    visited = set([start])
    max_distance = 0

    while queue:
        (row, col), distance = queue.popleft()
        max_distance = max(max_distance, distance)
        current_pipe = grid[row][col]

        for direction in connections[current_pipe]:
            d_row, d_col = offsets[direction]
            next_row, next_col = row + d_row, col + d_col

            if 0 <= next_row < len(grid) and 0 <= next_col < len(grid[0]) and \
               (next_row, next_col) not in visited and \
               is_valid_move(current_pipe, grid[next_row][next_col], direction):

                queue.append(((next_row, next_col), distance + 1))
                visited.add((next_row, next_col))

    return max_distance, visited

In [6]:
grid = parse_input(input)
max_distance, visited = bfs_furthest_distance(grid, find_start_position(grid))
max_distance

6806

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is <em>within the area enclosed by the loop</em>?</p>
<p>To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
</code></pre>
<p>The above loop encloses merely <em>four tiles</em> - the two pairs of <code>.</code> in the southwest and southeast (marked <code>I</code> below). The middle <code>.</code> tiles (marked <code>O</code> below) are <em>not</em> in the loop. Here is the same loop again with those regions marked:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||<em>OOOOO</em>||.
.||<em>OOOOO</em>||.
.|L-7<em>O</em>F-J|.
.|<em>II</em>|<em>O</em>|<em>II</em>|.
.L--J<em>O</em>L--J.
.....<em>O</em>.....
</code></pre>
<p>In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, <code>I</code> is still within the loop and <code>O</code> is still outside the loop:</p>
<pre><code>..........
.S------7.
.|F----7|.
.||<em>OOOO</em>||.
.||<em>OOOO</em>||.
.|L-7F-J|.
.|<em>II</em>||<em>II</em>|.
.L--JL--J.
..........
</code></pre>
<p>In both of the above examples, <code><em>4</em></code> tiles are enclosed by the loop.</p>
<p>Here's a larger example:</p>
<pre><code>.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
</code></pre>
<p>The above sketch has many random bits of ground, some of which are in the loop (<code>I</code>) and some of which are outside it (<code>O</code>):</p>
<pre><code><em>O</em>F----7F7F7F7F-7<em>OOOO</em>
<em>O</em>|F--7||||||||FJ<em>OOOO</em>
<em>O</em>||<em>O</em>FJ||||||||L7<em>OOOO</em>
FJL7L7LJLJ||LJ<em>I</em>L-7<em>OO</em>
L--J<em>O</em>L7<em>III</em>LJS7F-7L7<em>O</em>
<em>OOOO</em>F-J<em>II</em>F7FJ|L7L7L7
<em>OOOO</em>L7<em>I</em>F7||L7|<em>I</em>L7L7|
<em>OOOOO</em>|FJLJ|FJ|F7|<em>O</em>LJ
<em>OOOO</em>FJL-7<em>O</em>||<em>O</em>||||<em>OOO</em>
<em>OOOO</em>L---J<em>O</em>LJ<em>O</em>LJLJ<em>OOO</em>
</code></pre>
<p>In this larger example, <code><em>8</em></code> tiles are enclosed by the loop.</p>
<p>Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>Here are just the tiles that are <em>enclosed by the loop</em> marked with <code>I</code>:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ<em>I</em>F7FJ-
L---JF-JLJ<em>IIII</em>FJLJJ7
|F|F-JF---7<em>III</em>L7L|7|
|FFJF7L7F-JF7<em>II</em>L---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>In this last example, <code><em>10</em></code> tiles are enclosed by the loop.</p>
<p>Figure out whether you have time to search for the nest by calculating the area within the loop. <em>How many tiles are enclosed by the loop?</em></p>
</article>

In [7]:
def count_enclosed_tiles(grid: list[str], loop_pipes: set[tuple[int, int]]) -> int:
    """
    Count the number of tiles enclosed within the loop using a ray-casting approach.

    Args:
    grid (list[str]): The grid representing the pipe layout.
    loop_pipes (set[tuple[int, int]]): Set of tuples containing the row and column indices of the pipes in the loop.

    Returns:
    int: The number of tiles enclosed within the loop.
    """

    enclosed_count = 0
    for row in range(len(grid)):
        intercepts = 0
        corner_start = ''
        for col in range(len(grid[row])):
            if (row, col) in loop_pipes:
                pipe = grid[row][col]
                if pipe == '|':
                    intercepts += 1
                if pipe == 'F':
                    corner_start = 'F'
                if pipe == 'L':
                    corner_start = 'L'
                if pipe == 'J':
                    if corner_start == 'F':
                        intercepts += 1
                    corner_start = ''
                if pipe == '7':
                    if corner_start == 'L':
                        intercepts += 1
                    corner_start = ''
            else:
                if intercepts % 2 == 1:
                    enclosed_count += 1

    return enclosed_count

In [8]:
count_enclosed_tiles(grid, visited)

449